# 1. 데코레이터 기본 지식

In [7]:
def document_it(func):
    def new_function(*args, **kwargs):
        print('Running function:', func.__name__)
        print('Positional arguments:', args)
        print('Keyword arguments:', kwargs)
        result = func(*args, **kwargs)
        print('Result:', result)
        return result
    return new_function

In [8]:
def add_ints(a, b):
    return a + b

In [9]:
cooler_add_inits = document_it(add_ints) # 데커레이터 수동 할당
cooler_add_inits(3, 5)
# Running function: add_ints
# Positional arguments: (3, 5) 
# Keyword arguments: {}
# Result: 8
# 8

Running function: add_ints
Positional arguments: (3, 5)
Keyword arguments: {}
Result: 8


8

In [10]:
@document_it # 데코레이터를 사용하고 싶은 함수에 그냥 ''@데코레이터_이름' 추가
def add_ints(a, b):
    return a + b

In [11]:
add_ints(3, 5)
# Running function: add_ints
# Positional arguments: (3, 5)
# Keyword arguments: {}
# Result: 8
# 8

Running function: add_ints
Positional arguments: (3, 5)
Keyword arguments: {}
Result: 8


8

In [13]:
def square_it(func):
    def new_function(*args, **kwargs):
        result = func(*args, **kwargs)
        return result * result
    return new_function

@document_it
@square_it
def add_ints(a,b):
    return a + b

# Running function: new_function
# Positional arguments: (3, 5)
# Keyword arguments: {}
# Result: 64
# 64

In [15]:
# ------------------------------------------------------

In [17]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [18]:
@deco
def target():
    print('running target()')

In [19]:
target()

running inner()


In [20]:
target

<function __main__.deco.<locals>.inner()>

# 2. 파이썬이 데코레이터를 실행하는 시점

In [153]:
registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()  # <9>

running register(<function f1 at 0x107a3d750>)
running register(<function f2 at 0x107a3ca60>)
running main()
registry -> [<function f1 at 0x107a3d750>, <function f2 at 0x107a3ca60>]
running f1()
running f2()
running f3()


In [154]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [155]:
[item for item in dir(globals()) if '' in item]

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "def document_it(func):\n    def new_function(*args, **kwargs):\n        print('Running function:', func.__name__)\n        print('Positional arguments:', args)\n        print('Keyword arguments:', kwargs)\n        result = func(*args, **kwargs)\n        print('Result:', result)\n        return result\n    return new_function",
  'cooler_add_inits = document_it(add_ints) # 데커레이터 수동 할당\ncooler_add_inits(3, 5)\n# Running function: add_ints\n# Positional arguments: (3, 5) \n# Keyword arguments: {}\n# Result: 8\n# 8',
  "def document_it(func):\n    def new_function(*args, **kwargs):\n        print('Running function:', func.__name__)\n        print('Positional arguments:', args)\n        print('Keyword arguments:', k

# 3. 데코레이터로 개선한 전략 패턴

In [34]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [35]:
# BEGIN STRATEGY_BEST4

promos = []  # <1>

# promo_func를 promos 리스트에 추가한 후 그대로 반환.
# 얘가 어떻게 보면 인터페이스 역할? 관계없는 클래스들끼리 묶어주는 것처럼 할인 방식 빼먹지 않도록 얘도 묶어주는 역할..
def promotion(promo_func):  # <2>
    promos.append(promo_func)
    return promo_func


@promotion  # <3>
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [40]:
def best_promo(order):  # <4>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

In [41]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [42]:
cart = [LineItem('banana', 4, .5), 
       LineItem('apple', 10, 1.5),
       LineItem('watermelon', 5, 5.0)]

In [50]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

# 4. 변수 범위 규칙

대부분의 데코레이터는 데코레이트된 함수를 반환한다. 즉, 내부 함수를 정의하고 그것을 반환하여 데코레이트된 함수를 대체한다. <br>
내부 함수를 사용하는 코드는 제대로 작동하기 위해 거의 항상 클로저에 의존한다. <br>
클로저를 이해하기 위해 먼저 파이썬에서 `변수 범위의 작동 방식`에 대해 자세히 살펴보자

In [53]:
def f1(a):
    print(a)
    print(b)

In [54]:
f1(3)

3


NameError: name 'b' is not defined

In [55]:
b = 6

In [56]:
f1(3)

3
6


In [58]:
def f2(a):
    print(a)
    print(b)
    b = 9

In [60]:
f2(3) # 관련 내용 introducing python 10. 네임스페이스와 스코프에 있음
# 함수에서 전역 변수의 값을 1. 얻어서 2. 바꾸려고 하면 에러 발생.

3


UnboundLocalError: local variable 'b' referenced before assignment

In [65]:
def f3(a):
    global b
    
    print(a)
    print(b)
    b = 9

In [66]:
f3(3)

3
9


In [67]:
b = 30

In [68]:
b

30

In [70]:
f3(1)

1
30


In [71]:
f3(1)

1
9


In [72]:
from dis import dis

In [73]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [74]:
dis(f2)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# 5. 클로저

In [89]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [90]:
avg = Averager()

In [91]:
avg(10)

10.0

In [92]:
avg(11)

10.5

In [93]:
avg(12)

11.0

In [108]:
def make_averager():
    series = []

    def averager(new_value): # 호출될 때마다 받은 인수를 series에 추가
        print(hex(id(series)))
        series.append(new_value) # 함수 외부에 정의된 지역변수 참조
        total = sum(series)

        return total/len(series)
    
    return averager

In [109]:
avg = make_averager()

In [110]:
avg(10)

0x107ceb680


10.0

In [111]:
avg(11)

0x107ceb680


10.5

In [112]:
avg(12)

0x107ceb680


11.0

In [113]:
avg

<function __main__.make_averager.<locals>.averager(new_value)>

In [114]:
dir(avg)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [115]:
avg.__code__

<code object averager at 0x1079ab7e0, file "/var/folders/_s/0rymbx1n0m14zlxptddb3vrw0000gn/T/ipykernel_11435/1338861763.py", line 4>

In [116]:
avg.__code__.co_varnames

('new_value', 'total')

In [117]:
avg.__code__.co_freevars

('series',)

In [118]:
avg.__closure__ # 0x107ceb680

(<cell at 0x10794fd60: list object at 0x107ceb680>,)

In [123]:
dir(avg.__closure__[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'cell_contents']

In [124]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [127]:
hex(id(avg.__closure__))

'0x1078e4eb0'

# 6. nonlocal 선언

In [131]:
def make_average():
    count = 0
    total = 0
    
    def average(new_value):
        count += 1 # 얘가 값을 변경하는 문장이기 때문에 지역 변수로 보는데 그 지역변수가 없어서 발생하는 에러... 아까 위에서 봤던 거
        total += new_value
        
        return total / count
    
    return average

In [132]:
avg = make_average()

In [133]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [134]:
def make_average():
    count = 0
    total = 0
    
    def average(new_value):
        nonlocal count, total
        
        count += 1 
        total += new_value
        
        return total / count
    
    return average

In [136]:
avg = make_average()

In [138]:
avg(10)

10.0

In [139]:
avg(20)

13.333333333333334

# 7. 간단한 데코레이터 구현하기

In [4]:
import time

In [44]:
def clock(func):
    def clocked(*args): # 위치인수 받기
#         print('test23432423423 : ', *args)
        
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [45]:
@clock
def testFunc(numTest, hi):
    print(numTest)

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12804413s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000119s] factorial(1) -> 1
[0.00002098s] factorial(2) -> 2
[0.00003290s] factorial(3) -> 6
[0.00004506s] factorial(4) -> 24
[0.00005913s] factorial(5) -> 120
[0.00007415s] factorial(6) -> 720
6! = 720


In [46]:
hi = testFunc(1, 2)

1
[0.00004888s] testFunc(1, 2) -> None


In [47]:
print(hi)

None


In [48]:
testFunc(1, 2)

# 위치 인수에 저 별을 붙이는 이유가 뭐였더라.......

1
[0.00004387s] testFunc(1, 2) -> None


## 7.1 작동 과정

In [49]:
factorial

<function __main__.clock.<locals>.clocked(*args)>

In [50]:
factorial.__name__

'clocked'

# 8. 표준 라이브러리에서 제공하는 데코레이터

## 8.1 functools.lru_cache()를 이용한 메모이제이션

In [51]:
# 스칼라 서브쿼리 느낌..

In [52]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return 

In [53]:
@clock  # <2>
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000072s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00026512s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00001097s] fibonacci(2) -> 1
[0.00002217s] fibonacci(3) -> 2
[0.00029898s] fibonacci(4) -> 3
[0.00000119s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00001097s] fibonacci(2) -> 1
[0.00002003s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00001001s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00001097s] fibonacci(2) -> 1
[0.00002623s] fibonacci(3) -> 2
[0.00005198s] fibonacci(4) -> 3
[0.00008726s] fibonacci(5) -> 5
[0.00040174s] fibonacci(6) -> 8
8


### 캐시를 이용한 더 빠른 구현

# 10. 매개변수화된 데코레이터

## 10.1 매개변수화된 등록 데코레이터

In [54]:
registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x10690a290>)
running register(active=True)->decorate(<function f2 at 0x1069081f0>)


In [55]:
f1()

running f1()


In [58]:
f2(3)

TypeError: f2() takes 0 positional arguments but 1 was given

In [59]:
dir()

['In',
 'Out',
 '_',
 '_36',
 '_37',
 '_38',
 '_39',
 '_40',
 '_41',
 '_49',
 '_50',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i25',
 '_i26',
 '_i27',
 '_i28',
 '_i29',
 '_i3',
 '_i30',
 '_i31',
 '_i32',
 '_i33',
 '_i34',
 '_i35',
 '_i36',
 '_i37',
 '_i38',
 '_i39',
 '_i4',
 '_i40',
 '_i41',
 '_i42',
 '_i43',
 '_i44',
 '_i45',
 '_i46',
 '_i47',
 '_i48',
 '_i49',
 '_i5',
 '_i50',
 '_i51',
 '_i52',
 '_i53',
 '_i54',
 '_i55',
 '_i56',
 '_i57',
 '_i58',
 '_i59',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'clock',
 'exit',
 'f1',
 'f2',
 'f3',
 'factorial',
 'fibonacci',
 'get_ipython',
 'hi',
 'open',
 'print_args',
 'quit',
 'register',
 'registry',
 'snooze',
 'testFunc',
 'time']

In [60]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import time',
  "def clock(func):\n    def clocked(*args):\n        t0 = time.time()\n        result = func(*args)\n        elapsed = time.time() - t0\n        name = func.__name__\n        arg_str = ', '.join(repr(arg) for arg in args)\n        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))\n        return result\n    return clocked",
  "@clock\ndef snooze(seconds):\n    time.sleep(seconds)\n\n@clock\ndef factorial(n):\n    return 1 if n < 2 else n*factorial(n-1)\n\nif __name__=='__main__':\n    print('*' * 40, 'Calling snooze(.123)')\n    snooze(.123)\n    print('*' * 40, 'Calling factorial(6)')\n    print('6! =', factorial(6))",
  'import time',
  "def clock(func):\n    def clocked(*args):

In [71]:
class Gizmo:
    def __init__(self):
        print('Gizm')
    
    def __mul__(self, inta):
#         [for i in range(10)]
        print('test')
        return for self Gizmo *  

In [72]:
x = Gizmo()

Gizm


In [73]:
x

In [74]:
Gizmo() * 10

Gizm
test
10


In [76]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [83]:
charles = {'name': 'hi', 'age': 90}

In [84]:
alex = charles

In [85]:
alex is charles

True

In [86]:
alex['balanc'] = 950

In [87]:
alex

{'name': 'hi', 'age': 90, 'balanc': 950}

In [88]:
alex is charles

True

In [89]:
a = [10, 20]

In [90]:
b = [a, 30]

In [91]:
b

[[10, 20], 30]

In [92]:
a.append(b)

In [93]:
a

[10, 20, [[...], 30]]

In [94]:
type(1)

int

In [95]:
def f(a,b):
    print(id(a))
    a+=b
    print(id(a))
    return a
x=1
y=2
f(x,y)

4302225648
4302225712


3

In [96]:
x

1

In [97]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # <1>
        self.passengers = passengers  # <2>

    def pick(self, name):
        self.passengers.append(name)  # <3>

    def drop(self, name):
        self.passengers.remove(name)

In [101]:
type(range(10))

range

In [99]:
type(HauntedBus.__init__.__defaults__)

tuple

In [100]:
HauntedBus.__init__.__defaults__[0]

[]